In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data

df=pd.read_csv('SpamBase_withoutdupl_norm_40.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att49,att50,att51,att52,att53,att54,att55,att56,att57,outlier
0,0.000000,0.044818,0.125490,0.0,0.032,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.023955,0.000000,0.000000,0.002502,0.006007,0.017487,1
1,0.046256,0.019608,0.098039,0.0,0.014,0.047619,0.028886,0.006301,0.000000,0.051705,...,0.000000,0.013536,0.0,0.011454,0.029985,0.002421,0.003735,0.010012,0.064836,1
2,0.013216,0.000000,0.139216,0.0,0.123,0.032313,0.026135,0.010801,0.121673,0.013751,...,0.002281,0.014664,0.0,0.008498,0.030651,0.000504,0.008008,0.048458,0.142551,1
3,0.000000,0.000000,0.000000,0.0,0.063,0.000000,0.042641,0.056706,0.058935,0.034653,...,0.000000,0.014048,0.0,0.004218,0.000000,0.000000,0.002303,0.003905,0.011995,1
4,0.000000,0.000000,0.000000,0.0,0.063,0.000000,0.042641,0.056706,0.058935,0.034653,...,0.000000,0.013843,0.0,0.004157,0.000000,0.000000,0.002303,0.003905,0.011995,1


In [3]:
df.shape

(4207, 58)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/SpamBase.json")
gc = GCForest(config)

In [15]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,57]
X = df[:,0:57]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
            # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 00:59:35,236][cascade_classifier.fit_transform] X_groups_train.shape=[(2818, 57)],y_train.shape=(2818,),X_groups_test.shape=[(1389, 57)],y_test.shape=(1389,)
[ 2018-04-25 00:59:35,238][cascade_classifier.fit_transform] group_dims=[57]
[ 2018-04-25 00:59:35,240][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:59:35,242][cascade_classifier.fit_transform] group_ends=[57]
[ 2018-04-25 00:59:35,244][cascade_classifier.fit_transform] X_train.shape=(2818, 57),X_test.shape=(1389, 57)
[ 2018-04-25 00:59:35,246][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2818, 57), X_cur_test.shape=(1389, 57)
[ 2018-04-25 00:59:36,007][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.05%
[ 2018-04-25 00:59:36,732][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.29%
[ 2018-04-25 00:59:37,460][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:59:43,028][cascade_classifier.transform] X_groups_test.shape=[(1389, 57)]
[ 2018-04-25 00:59:43,033][cascade_classifier.transform] group_dims=[57]
[ 2018-04-25 00:59:43,034][cascade_classifier.transform] X_test.shape=(1389, 57)
[ 2018-04-25 00:59:43,036][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1389, 57)


Test Accuracy of ExtraTrees = 95.176386 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 00:59:44,583][cascade_classifier.transform] X_groups_test.shape=[(1389, 57)]
[ 2018-04-25 00:59:44,586][cascade_classifier.transform] group_dims=[57]
[ 2018-04-25 00:59:44,587][cascade_classifier.transform] X_test.shape=(1389, 57)
[ 2018-04-25 00:59:44,590][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1389, 57)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 00:59:45,987][cascade_classifier.fit_transform] X_groups_train.shape=[(2818, 59)],y_train.shape=(2818,),X_groups_test.shape=[(1389, 59)],y_test.shape=(1389,)
[ 2018-04-25 00:59:45,991][cascade_classifier.fit_transform] group_dims=[59]
[ 2018-04-25 00:59:45,991][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:59:45,993][cascade_classifier.fit_transform] group_ends=[59]
[ 2018-04-25 00:59:45,994][cascade_classifier.fit_transform] X_train.shape=(2818, 59),X_test.shape=(1389, 59)
[ 2018-04-25 00:59:45,997][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2818, 59), X_cur_test.shape=(1389, 59)
[ 2018-04-25 00:59:46,578][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.76%
[ 2018-04-25 00:59:47,336][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.64%
[ 2018-04-25 00:59:48,110][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:59:53,555][cascade_classifier.transform] X_groups_test.shape=[(1389, 59)]
[ 2018-04-25 00:59:53,557][cascade_classifier.transform] group_dims=[59]
[ 2018-04-25 00:59:53,560][cascade_classifier.transform] X_test.shape=(1389, 59)
[ 2018-04-25 00:59:53,563][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1389, 59)


Test Accuracy of ExtraTrees = 95.104392 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 00:59:55,174][cascade_classifier.fit_transform] X_groups_train.shape=[(2818, 59)],y_train.shape=(2818,),X_groups_test.shape=[(1389, 59)],y_test.shape=(1389,)
[ 2018-04-25 00:59:55,176][cascade_classifier.fit_transform] group_dims=[59]
[ 2018-04-25 00:59:55,177][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:59:55,179][cascade_classifier.fit_transform] group_ends=[59]
[ 2018-04-25 00:59:55,181][cascade_classifier.fit_transform] X_train.shape=(2818, 59),X_test.shape=(1389, 59)
[ 2018-04-25 00:59:55,183][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2818, 59), X_cur_test.shape=(1389, 59)
[ 2018-04-25 00:59:55,807][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.76%
[ 2018-04-25 00:59:56,547][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.29%
[ 2018-04-25 00:59:57,339][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 94.960403 %', 0, 'fois')


[ 2018-04-25 01:00:05,439][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.76%
[ 2018-04-25 01:00:06,283][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.64%
[ 2018-04-25 01:00:07,053][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=95.39%
[ 2018-04-25 01:00:07,843][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=96.45%
[ 2018-04-25 01:00:08,636][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=94.68%
[ 2018-04-25 01:00:09,434][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=95.04%
[ 2018-04-25 01:00:10,246][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=95.37%
[ 2018-04-25 01:00:11,147][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=96.09%


('Test Accuracy of ExtraTrees = 94.960403 %', 1, 'fois')


[ 2018-04-25 01:00:15,169][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=95.05%
[ 2018-04-25 01:00:15,975][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=92.93%
[ 2018-04-25 01:00:16,764][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=93.97%
[ 2018-04-25 01:00:17,558][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=94.68%
[ 2018-04-25 01:00:18,389][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=93.97%
[ 2018-04-25 01:00:19,182][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=94.68%
[ 2018-04-25 01:00:20,040][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=94.31%
[ 2018-04-25 01:00:20,856][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=96.09%


...

('Test Accuracy of ExtraTrees = 94.888409 %', 17, 'fois')


[ 2018-04-25 01:02:51,929][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=93.99%
[ 2018-04-25 01:02:52,712][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=93.64%
[ 2018-04-25 01:02:53,506][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=93.97%
[ 2018-04-25 01:02:54,365][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=95.39%
[ 2018-04-25 01:02:55,187][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=94.68%
[ 2018-04-25 01:02:55,998][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=93.62%
[ 2018-04-25 01:02:56,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=95.37%
[ 2018-04-25 01:02:57,634][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=96.09%


('Test Accuracy of ExtraTrees = 94.600432 %', 18, 'fois')


[ 2018-04-25 01:03:01,684][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=93.64%
[ 2018-04-25 01:03:02,475][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=92.93%
[ 2018-04-25 01:03:03,251][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=94.68%
[ 2018-04-25 01:03:04,086][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=95.39%
[ 2018-04-25 01:03:04,868][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=95.04%
[ 2018-04-25 01:03:05,662][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=93.26%
[ 2018-04-25 01:03:06,480][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=95.02%
[ 2018-04-25 01:03:07,362][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=96.09%


('Test Accuracy of ExtraTrees = 94.744420 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 95.248380 %', 3, 'fois')